In [ ]:
!apt-get update && apt-get install -y python3-opencv
%pip install scikit-learn scipy matplotlib

%pip install -q tensorflow-addons
%pip install -q tensorflow-probability
%pip install -q opencv-python-headless
%pip install -q seaborn


%pip install -qU wandb
%pip install -qU scikit-learn

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf

import os
import tensorflow_datasets as tfds
#import tensorflow_addons as tfa
import tensorflow_probability as tfp

# CHANGED FOR TPU 1VM:
# Detect hardware, return appropriate distribution strategy
'''try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect(tpu="local") # "local" for 1VM TPU
    strategy = tf.distribute.TPUStrategy(tpu)
except tf.errors.NotFoundError:
    strategy = tf.distribute.MirroredStrategy()
    
print("REPLICAS: ", strategy.num_replicas_in_sync)
'''
#resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
#tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
#tf.tpu.experimental.initialize_tpu_system(resolver)
#print("All devices: ", tf.config.list_logical_devices('TPU'))

In [ ]:
import cv2
def feature(image):

    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    return image

In [ ]:
height = 512
width = 512
font_size = 20
def apply_visual_attention(path):
    img = cv2.imread(path, 0)
#     img = cv2.imread(path)
#     resized_img = cv2.resize(img, (height, width))
#     denoised_img = cv2.medianBlur(resized_img, 5)
#     th = cv2.adaptiveThreshold(denoised_img, maxValue = 255, adaptiveMethod = cv2.ADAPTIVE_THRESH_GAUSSIAN_C, thresholdType = cv2.THRESH_BINARY, blockSize = 11, C = 2)
    return img

In [ ]:

import os
import cv2
import numpy as np

# Set the path to the dataset folders
train_image_path = '/kaggle/input/cod10k/COD10K-v3/Train/Image/'
train_gt_path = '/kaggle/input/cod10k/COD10K-v3/Train/GT_Object/'
test_image_path = '/kaggle/input/cod10k/COD10K-v3/Test/Image/'
test_gt_path = '/kaggle/input/cod10k/COD10K-v3/Test/GT_Object/'


img_size2=(512,512)


# Function to load images and ground truth instances
def load_data(image_path, gt_path,maxi):
    images = []
    gt_instances = []
    c=0
    for filename in sorted(os.listdir(image_path)):
        # Check if file is a JPG image
        c+=1
        if(c>=maxi):
            break
        if(c%50==0):
            print(c)
        if filename.endswith('.jpg'):
            # Load image and resize to (256,256)
#             img = cv2.imread(image_path + filename)
            img=apply_visual_attention(image_path + filename)
            img = cv2.resize(img, img_size2)
            images.append(img)
            # Load ground truth instance and resize to (256,256)
            gt = cv2.imread(gt_path + filename[:-4] + '.png', cv2.IMREAD_GRAYSCALE)
            gt = cv2.resize(gt, img_size2, interpolation=cv2.INTER_LINEAR)
            gt_instances.append(gt)
    return np.array(images), np.array(gt_instances)


# Load training data
train_images, train_gt_instances = load_data(train_image_path, train_gt_path,6000)

# Load testing data
test_images, test_gt_instances = load_data(test_image_path, test_gt_path,2000)

print('done')


In [ ]:
print('hello')

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dropout
# from tensorflow.keras.optimizers import Adam

# from tensorflow.keras import backend
# backend.clear_session()


# def build_generator(input_shape):
#     model = Sequential()
#     # Encoder
#     model.add(Conv2D(64, kernel_size=(3, 3), strides=(2, 2), input_shape=input_shape, padding='same', activation='relu'))
#     model.add(Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'))
#     model.add(Conv2D(256, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'))
#     model.add(Dropout(0.5))
#     # Decoder
#     model.add(Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', activation='relu'))
#     model.add(Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu'))
#     model.add(Conv2DTranspose(1, (3, 3), strides=(2, 2), padding='same', activation='sigmoid'))
    
#     # Output
#     model.add(Conv2D(1, (1, 1), activation='sigmoid'))

#     return model



# with strategy.scope():
#     input_shape = (512,512,3)
#     generator = build_generator(input_shape)
    
#     # Define optimizer and loss function
#     optimizer = tf.keras.optimizers.Adam(lr=0.001)
#     loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

#     # Compile the model
#     generator.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
    
#     generator.summary()




import tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dropout
from tensorflow.keras.optimizers import Adam

from tensorflow.keras import backend
backend.clear_session()


def build_generator(input_shape):
    model = Sequential()
    # Encoder
    model.add(Conv2D(64, kernel_size=(3, 3), strides=(2, 2), input_shape=input_shape, padding='same', activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(Conv2D(256, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(Conv2D(512, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(Dropout(0.5))
    # Decoder
    model.add(Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(Conv2DTranspose(1, (3, 3), strides=(2, 2), padding='same', activation='sigmoid'))
    
    # Output
    model.add(Conv2D(1, (1, 1), activation='sigmoid'))

    return model



with strategy.scope():
    input_shape = (512,512,1)
    generator = build_generator(input_shape)
    
    # Define optimizer and loss function
    optimizer = tf.keras.optimizers.Adam(lr=0.001)
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

    # Compile the model
    generator.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy', tf.keras.metrics.Precision()])
    
    generator.summary()









# #---- managable output with all images (520,720,  3) for 50 epochs, bsize 32

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dropout
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras import backend
# from tensorflow.keras.losses import binary_crossentropy

# backend.clear_session()

# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, UpSampling2D

# # Define the U-Net model
# def unet(input_shape=(512, 512, 3)):
#     inputs = Input(input_shape)
    
#     # Encoder
#     conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
#     conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
#     pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
#     conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
#     conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
#     pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
#     conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
#     conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
#     pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
#     # Decoder
#     up4 = UpSampling2D(size=(2, 2))(pool3)
#     conv4 = Conv2D(256, 2, activation='relu', padding='same')(up4)
#     merge4 = concatenate([conv3, conv4], axis=3)
#     conv4 = Conv2D(256, 3, activation='relu', padding='same')(merge4)
#     conv4 = Conv2D(256, 3, activation='relu', padding='same')(conv4)
#     up5 = UpSampling2D(size=(2, 2))(conv4)
#     conv5 = Conv2D(128, 2, activation='relu', padding='same')(up5)
#     merge5 = concatenate([conv2, conv5], axis=3)
#     conv5 = Conv2D(128, 3, activation='relu', padding='same')(merge5)
#     conv5 = Conv2D(128, 3, activation='relu', padding='same')(conv5)
#     up6 = UpSampling2D(size=(2, 2))(conv5)
#     conv6 = Conv2D(64, 2, activation='relu', padding='same')(up6)
#     merge6 = concatenate([conv1, conv6], axis=3)
#     conv6 = Conv2D(64, 3, activation='relu', padding='same')(merge6)
#     conv6 = Conv2D(64, 3, activation='relu', padding='same')(conv6)

#     # Output layer
#     outputs = Conv2D(3, 1, activation='sigmoid')(conv6)

#     # Define the model
#     model = Model(inputs=inputs, outputs=outputs)

#     return model



# with strategy.scope():
#     input_shape = (512,512, 3)
#     generator = unet(input_shape)
    
#     optimizer = tf.keras.optimizers.Adam(lr=0.001)
    
#     loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

#     generator.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy', tf.keras.metrics.Precision()])

#     generator.summary()








In [ ]:
train_images[0].shape

In [ ]:
train_gt_instances[0].shape

In [ ]:
num_epochs = 25
batch_size = 32

# Train the model
history = generator.fit(train_images, train_gt_instances,
                        epochs=num_epochs,
                        batch_size=batch_size,
                        validation_data=(test_images, test_gt_instances),
#                         reset_metrics=True
                       )


In [ ]:
# import numpy as np
# from keras.models import Model
# from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, BatchNormalization, Activation

# def residual_block(input_layer, num_filters, kernel_size=(3,3), strides=(1,1), padding='same'):
#     x = Conv2D(num_filters, kernel_size, strides=strides, padding=padding)(input_layer)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     x = Conv2D(num_filters, kernel_size, strides=strides, padding=padding)(x)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     shortcut = Conv2D(num_filters, kernel_size=(1,1), strides=strides, padding=padding)(input_layer)
#     shortcut = BatchNormalization()(shortcut)
#     x = concatenate([x, shortcut])
#     x = Activation('relu')(x)
#     return x

# def sinet(input_shape=(256, 256, 3)):
#     inputs = Input(input_shape)

#     # Encoding Path
#     conv1 = residual_block(inputs, 64)
#     pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

#     conv2 = residual_block(pool1, 128)
#     pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

#     conv3 = residual_block(pool2, 256)
#     pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

#     # Decoding Path
#     up4 = UpSampling2D(size=(2, 2))(pool3)
#     conv4 = residual_block(up4, 256)
#     concat4 = concatenate([conv4, conv3], axis=-1)

#     up5 = UpSampling2D(size=(2, 2))(concat4)
#     conv5 = residual_block(up5, 128)
#     concat5 = concatenate([conv5, conv2], axis=-1)

#     up6 = UpSampling2D(size=(2, 2))(concat5)
#     conv6 = residual_block(up6, 64)
#     concat6 = concatenate([conv6, conv1], axis=-1)

#     # Output
#     outputs = Conv2D(1, (1, 1), activation='sigmoid')(concat6)

#     # Model definition
#     model = Model(inputs=[inputs], outputs=[outputs])
#     return model

# from tensorflow.keras import backend
# backend.clear_session()

# with strategy.scope():
#     input_shape = (256,256,1)
#     generator = sinet(input_shape)

#     # Define optimizer and loss function
#     optimizer = tf.keras.optimizers.Adam(lr=0.0005)
#     loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

#     # Compile the model
#     generator.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy', tf.keras.metrics.Precision()])
# #     generator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#     generator.summary()

In [ ]:
num_epochs = 100
batch_size = 64

# Train the model
history = generator.fit(train_images, train_gt_instances,
                        epochs=num_epochs,
                        batch_size=batch_size,
                        validation_data=(test_images, test_gt_instances),
#                         reset_metrics=True
                       )


In [ ]:
history2 = generator.fit(train_images, train_gt_instances,
                        epochs=num_epochs,
                        batch_size=batch_size,
                        validation_data=(test_images, test_gt_instances),
#                         reset_metrics=True
                       )


In [ ]:

import matplotlib.pyplot as plt
img_path='/kaggle/input/cod10k/COD10K-v3/Train/Image/COD10K-CAM-1-Aquatic-12-Pagurian-480.jpg'
otimg = cv2.imread(img_path)
otimg=apply_visual_attention(img_path)
timg = cv2.resize(otimg, img_size2)


plt.imshow(timg)

timg=np.array([timg])
print(timg.shape)
predictions = generator.predict(timg)

In [ ]:
plt.imshow(predictions[0])

In [ ]:
# timg='/kaggle/input/cod10k/COD10K-v3/Test/Image/COD10K-CAM-1-Aquatic-13-Pipefish-528.jpg'

timg= img_path.split('/')

timgp=timg.pop().split('.')
timgp=[timgp[0],'png']
timgp=".".join(timgp)
timg[6]='GT_Object'

timg=timg+[timgp]
timg='/'.join(timg)
print(timg)
timg = cv2.resize(cv2.imread(timg), img_size2)
# timg=feature(timg)
plt.imshow(timg)

In [ ]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt

# # Load the image
# img = cv2.imread('/kaggle/input/cod10k/COD10K-v3/Train/Image/COD10K-CAM-1-Aquatic-1-BatFish-1.jpg')

# # Convert the image to the LMS color space
# lms = cv2.cvtColor(img, cv2.COLOR_BGR2LMS)

# # Define the range of colors to extract (in LMS)
# lower_range = np.array([0.1, 0.1, 0.1])
# upper_range = np.array([0.9, 0.9, 0.9])

# # Threshold the image to extract the specified colors
# mask = cv2.inRange(lms, lower_range, upper_range)

# # Compute the color histogram
# hist = cv2.calcHist([img], [0], mask, [256], [0, 256])

# # Normalize the histogram
# hist = cv2.normalize(hist, None)

# # Create a black image
# hist_img = np.zeros((256, 256, 3), np.uint8)

# # Draw the histogram on the image
# for i in range(256):
#     x1 = i
#     y1 = 255
#     x2 = i
#     y2 = 255 - int(hist[i] * 255)
#     cv2.line(hist_img, (x1, y1), (x2, y2), (255, 255, 255), 1)

# # Display the histogram image
# plt.imshow('Histogram')
# cv2.destroyAllWindows()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16

# Clear backend session
from tensorflow.keras import backend
backend.clear_session()

# Set input shape for images
input_shape = (512, 512, 3)

# Define function to build generator model
def build_generator(input_shape):
    # Load pre-trained VGG16 model for feature extraction
    vgg16_model = VGG16(input_shape=input_shape, include_top=False, weights='imagenet')
    vgg16_model.trainable = False

    # Create generator model
    model = Sequential()
    model.add(vgg16_model)
    model.add(Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(Conv2DTranspose(1, (3, 3), strides=(2, 2), padding='same', activation='sigmoid'))
    model.add(Conv2D(1, (1, 1), activation='sigmoid'))

    return model

# Define function to build discriminator model
def build_discriminator(input_shape):
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3, 3), strides=(2, 2), input_shape=input_shape, padding='same', activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(Conv2D(256, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

# Define function to train the GAN
def train_gan(generator, discriminator, combined, x_train, batch_size, epochs):
    # Define loss functions
    adversarial_loss_fn = tf.keras.losses.BinaryCrossentropy()
    generator_loss_fn = tf.keras.losses.MeanSquaredError()

    # Define optimizers
    adversarial_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

    # Train the GAN
    for epoch in range(epochs):
        # Train discriminator
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        real_images = x_train[idx]
        fake_images = generator.predict(real_images)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(17, 7))

# define interval ranges for hue, saturation, and lightness
hue_ranges = [(0, 270), (20, 45), (45, 55), (55, 80), (80, 100), (100, 120),
            (120, 140), (140, 165), (165, 190), (190, 220), (220, 255), (255, 270)]
sat_ranges = [(0.0, 0.1), (0.1, 0.25), (0.25, 0.45), (0.45, 0.65), (0.65, 0.85), (0.85, 1)]
val_ranges = [(0.0, 0.15), (0.15, 0.4), (0.4, 0.6), (0.6, 0.8), (0.8, 1.0)]

# define function to quantize HSV image based on interval ranges
def quantize_hsv(img):
    # convert image to HSV color space
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # quantize hue, saturation, and lightness separately
    h, s, v = cv2.split(hsv_img)
    h_quantized = np.zeros_like(h)
    s_quantized = np.zeros_like(s)
    v_quantized = np.zeros_like(v)
    for i, (h_range, s_range, v_range) in enumerate(zip(hue_ranges, sat_ranges, val_ranges)):
        h_quantized[((h >= h_range[0]) & (h < h_range[1]))] = i
        s_quantized[((s >= s_range[0]) & (s < s_range[1]))] = i
        v_quantized[((v >= v_range[0]) & (v < v_range[1]))] = i
    # combine quantized hue, saturation, and lightness to form Q
    Q = 25 * h_quantized + 5 * s_quantized + v_quantized
    return Q

# define function to generate color histogram of an image
def color_histogram(img):
    # quantize image using HSV color space
    Q = quantize_hsv(img)
    
    # showing image
    fig.add_subplot(1, 2, 1)
    plt.imshow(Q)
    plt.axis('off')
    plt.title("Quantize_Image_Output")
    
    # compute color histogram of quantized image
    hist, _ = np.histogram(Q.ravel(), bins=range(301))
    # normalize histogram
    hist = hist.astype(float) / float(img.size)
    return hist

# load input image
img = cv2.imread('/kaggle/input/cod10k/COD10K-v3/Train/Image/COD10K-CAM-1-Aquatic-1-BatFish-1.jpg')

# generate color histogram of input image
hist = color_histogram(img)

Q = quantize_hsv(img)

# Adds a subplot at the 2nd position
fig.add_subplot(1, 2, 2)
# plot histogram
plt.bar(range(300), hist)
plt.title("Histogram of Quantized Image")
plt.show()

In [ ]:
import cv2
import numpy as np

# define the increment function of lightness visual attention
def fv(v, kv=1.0):
    return kv * v**0.6

# define the increment function of hue visual attention
def fh(h):
    if np.mod(h, np.pi/3) == 0:
        return 1
    else:
        return np.cos(np.mod(h, np.pi/3) - (np.pi/6)) / np.cos(np.pi/6)

# define the increment function of chroma visual attention
def fc(h, s, v, kv=1.0, kc=1.0):
    return kv * fv(v, kv) * fh(h) * s**0.6 * kc

# define the function of color visual attention
def f(h, s, v, kv=1.0, kc=1.0):
    return 1 + fv(v, kv) + fc(h, s, v, kv, kc)


# convert the image to HSV color space
hsv_img = img

# initialize an empty matrix for the color visual attention map
cva_map = np.zeros_like(hsv_img[:, :, 0], dtype=float)

# iterate over each pixel in the image and calculate its color visual attention
for i in range(hsv_img.shape[0]):
    for j in range(hsv_img.shape[1]):
        h, s, v = hsv_img[i, j]
        cva = f(h, s, v)
        cva_map[i, j] = cva

# normalize the color visual attention map to the range [0, 255]
cva_map = cv2.normalize(cva_map, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

# display the color visual attention map
plt.imshow(cv2.applyColorMap(cva_map, cv2.COLORMAP_JET))

In [ ]:
#import cv2
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(17, 7))


# define function to generate color histogram of an image
def color_histogram_percp(img):
    # showing image
    fig.add_subplot(1, 2, 1)
    plt.imshow(cva_map)
    plt.axis('off')
    plt.title("CVA Image")
    
    # compute color histogram of quantized image
    hist, _ = np.histogram(cva_map.ravel(), bins=range(301))
    # normalize histogram
    hist = hist.astype(float) / float(img.size)
    return hist

hist2 = color_histogram_percp(img)

# Adds a subplot at the 2nd position
fig.add_subplot(1, 2, 2)
# plot histogram
plt.bar(range(300), hist2)
plt.title("Histogram of CVA Image")
plt.show()

In [ ]:
import cv2
import numpy as np

def calculate_color_feature(image, mask):
    # convert the image to the HSV color space
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # define the visual attention function
    def f(h, s, v):
        return np.exp(-(h / 16)**2) * np.exp(-(s / 255)**2) * np.exp(-(v / 255)**2)
    # calculate the color histogram
    color_hist = np.zeros((32768))
    visual_attention = np.zeros((128, 16, 16))
    for i in range(0, 256, 8):
        for j in range(0, 256, 8):
            for k in range(0, 256, 8):
                h, s, v = cv2.split(cv2.merge([np.array([i]), np.array([j]), np.array([k])]))
                ci = i * 25 + j * 5 + k
                visual_attention[ci] = f(h, s, v)
    # normalize the color histogram based on visual perception
    visual_attention = visual_attention / visual_attention.sum()
    color_hist = visual_attention.reshape(-1) * color_hist
    # calculate the color feature of the region Dk
    h = np.sum(color_hist * hsv_img[:, :, 0][mask]) / np.sum(color_hist)
    s = np.sum(color_hist * hsv_img[:, :, 1][mask]) / np.sum(color_hist)
    v = np.sum(color_hist * hsv_img[:, :, 2][mask]) / np.sum(color_hist)
    return (h, s, v)



# create the mask for the region Dk
mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
mask[100:200, 100:200] = 255

# calculate the color feature of the region Dk
color_feature = calculate_color_feature(img, mask)

# display the color feature
print("Color feature:", color_feature)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(17, 7))

# define interval ranges for hue, saturation, and lightness
hue_ranges = [(0, 270), (20, 45), (45, 55), (55, 80), (80, 100), (100, 120), 
              (120, 140), (140, 165), (165, 190), (190, 220), (220, 255), (255, 270)]
sat_ranges = [(0.0, 0.1), (0.1, 0.25), (0.25, 0.45), (0.45, 0.65), (0.65, 0.85), (0.85, 1)]
val_ranges = [(0.0, 0.15), (0.15, 0.4), (0.4, 0.6), (0.6, 0.8), (0.8, 1.0)]

# define function to quantize HSV image based on interval ranges
def quantize_hsv(img):
    # convert image to HSV color space
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # quantize hue, saturation, and lightness separately
    h, s, v = cv2.split(hsv_img)
    h_quantized = np.zeros_like(h)
    s_quantized = np.zeros_like(s)
    v_quantized = np.zeros_like(v)
    for i, (h_range, s_range, v_range) in enumerate(zip(hue_ranges, sat_ranges, val_ranges)):
        h_quantized[((h >= h_range[0]) & (h < h_range[1]))] = i
        s_quantized[((s >= s_range[0]) & (s < s_range[1]))] = i
        v_quantized[((v >= v_range[0]) & (v < v_range[1]))] = i
    # combine quantized hue, saturation, and lightness to form Q
    Q = 25 * h_quantized + 5 * s_quantized + v_quantized
    return Q, hsv_img

# define function to generate color histogram of an image
def color_histogram(img):
    # quantize image using HSV color space
    Q, _ = quantize_hsv(img)
    
    # showing image
    fig.add_subplot(1, 2, 1)
    plt.imshow(Q)
    plt.axis('off')
    plt.title("Quantize_Image_Output")
    
    # compute color histogram of quantized image
    hist, _ = np.histogram(Q.ravel(), bins=range(301))
    # normalize histogram
    hist = hist.astype(float) / float(img.size)
    return hist

# define the increment function of lightness visual attention
def fv(v, kv=1.0):
    return kv * v**0.6

# define the increment function of hue visual attention
def fh(h):
    if np.mod(h, np.pi/3) == 0:
        return 1
    else:
        return np.cos(np.mod(h, np.pi/3) - (np.pi/6)) / np.cos(np.pi/6)

# define the increment function of chroma visual attention
def f(h, s, v, kv=1.0, kc=1.0):
    return 1 + fv(v, kv) + fc(h, s, v, kv, kc)
for j in range(hsv_img.shape[1]):
    h, s, v = hsv_img[i, j]
    cva = f(h, s, v)
    cva_map[i, j] = cva

#normalize the color visual attention map to the range [0, 255]
cva_map = cv2.normalize(cva_map, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

#display the original image and the color visual attention map
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 10))
ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax1.set_title('Original Image')
ax1.axis('off')
ax2.imshow(cv2.applyColorMap(cva_map, cv2.COLORMAP_JET))
ax2.set_title('Color Visual Attention Map')
ax2.axis('off')
plt.show()


In [ ]:
%pip install tensorflow
# Import required libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.utils import load_img,img_to_array
from PIL import Image


def load_mask( image_path):
    # Load the original image
    img = Image.open(image_path)
    
    # Resize the image to the same shape as the input images
    img = img.resize((512,512), resample=Image.BILINEAR)
    
    # Convert the image to grayscale and then to a binary mask
    img = img.convert('L')
    img = np.array(img)
    img[img > 0] = 1
    img = np.expand_dims(img, axis=-1)
    
    return img

# Import the necessary module
from tensorflow.distribute import MirroredStrategy

# Create a MirroredStrategy.
strategy = MirroredStrategy()
with strategy.scope():
    # Define paths to train and test data
#     train_data_dir = '/kaggle/input/cod10k/COD10K-v3/Train/Image'
#     test_data_dir = '/kaggle/input/cod10k/COD10K-v3/Test/Image'

#     # Define path to ground truth file
#     train_ground_truth_dir = '/kaggle/input/cod10k/COD10K-v3/Train/GT_Object'
#     test_ground_truth_dir = '/kaggle/input/cod10k/COD10K-v3/Test/GT_Object'

    
#     # Define image parameters
    img_width, img_height = 512, 512
    input_shape = (img_width, img_height, 3)

#     # Define batch size and number of epochs
#     batch_size = 32
#     epochs = 10

#     # Define image generators with data augmentation for training set
#     train_datagen = ImageDataGenerator(
#         rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True)

#     # Define image generator without data augmentation for test set
#     test_datagen = ImageDataGenerator(rescale=1./255)

#     # Load original images and ground truth images separately for the training set
#     train_images = []
#     train_ground_truths = []

#     for file_name in os.listdir(train_data_dir):
#         # Load original image
#         img = load_img(os.path.join(train_data_dir, file_name), target_size=(img_width, img_height))
#         img_arr = img_to_array(img)
#         train_images.append(img_arr)

#         # Load ground truth image
#         gt_img = load_img(os.path.join(train_ground_truth_dir, file_name[:-4] + '.png'), target_size=(img_width, img_height), color_mode='grayscale')
#         gt_img_arr = img_to_array(gt_img)
#         gt_img_arr /= 255.0
#         train_ground_truths.append(gt_img_arr)

#     # Convert training data to numpy arrays
#     train_images = np.array(train_images)
#     train_ground_truths = np.array(train_ground_truths)

#     # Combine original images and ground truth images for the training set
#     train_generator = train_datagen.flow(train_images, train_ground_truths, batch_size=batch_size)

#     # Load original images and ground truth images separately for the test set
#     test_images = []
#     test_ground_truths = []

#     for file_name in os.listdir(test_data_dir):
#         # Load original image
# #         img = load_img(os.path.join(test_data_dir, file_name), target_size=(img_width, img_height))
# #         img_arr = img_to_array(img)
# #         test_images.append(img_arr)

# #         # Load ground truth image
# #         gt_img = load_img(os.path.join(test_ground_truth_dir, file_name[:-4] + '.png'), target_size=(img_width, img_height), color_mode='grayscale')
# #         gt_img_arr = img_to_array(gt_img)
# #         gt_img_arr /= 255.0
#         gt_img_arr=load_mask(os.path.join(test_data_dir, file_name))
#         test_ground_truths.append(gt_img_arr)

#     # Convert test data to numpy arrays
#     test_images = np.array(test_images)
#     test_ground_truths = np.array(test_ground_truths)

#     # Combine original images and ground truth images for the test set
#     test_generator = test_datagen.flow(test_images, test_ground_truths, batch_size=batch_size)

    %pip install tensorflow
    # Freeze the pre-trained weights of the base model
    from tensorflow.keras.applications import MobileNetV2

# Load MobileNetV2 as the base model
    base_model = MobileNetV2(weights='imagenet', include_top=False)
    base_model.trainable = False

    # Add a custom head on top of the base model for fine-tuning
    inputs = keras.Input(shape=input_shape)
    x = base_model(inputs, training=False)
#     x = layers.GlobalAveragePooling2D()(x)
#     x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
#     outputs = layers.Dense(1, activation='sigmoid')(x)
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)
    model = keras.Model(inputs, outputs)

    # Compile the model with binary cross-entropy loss and Adam optimizer
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    # Create an ImageDataGenerator object
    datagen = ImageDataGenerator(rescale=1./255)

    # Assuming train_dir is the path to the directory where your training images are
    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

    # Train the model on the train data
    model.fit(train_generator, epochs=epochs, validation_data=test_generator)
    model.summary()

history = model.fit(train_images, train_gt_instances,
                        epochs=10,
                        batch_size=32,
                        validation_data=(test_images, test_gt_instances),
                    )
